# Yolov8_Animeface

## A model trained to detect anime faces

We're using comet-ml to monitor the training, and the ultralytics library to download the model. Dataset is made on roboflow and is stored under the "dataset" subfolder of the project.

Dataset not provided in this document.


## Import Monitoring Services

In [ ]:
from comet_ml import Experiment

# Don't turn the code log on if you plan on pasting your api key here.

experiment = Experiment(
    api_key="",
    project_name="",
    workspace="",
)


## Train Model

In [ ]:
import torch
# Just making sure cuda is available, if not, check your driver installation.
torch.cuda.is_available()

In [ ]:
from ultralytics import YOLO

# Import the desired model. Look up ultralytics' docs to find out which one you should use. We will use the basic yolov8x here.
yolov8_animeface = YOLO('yolov8x6.pt')

results = yolov8_animeface.train(
    epochs=300,
    data='./dataset/data.yaml',
    imgsz=1280, # Feel free to tweak imgsz, but be careful as it will be more computally intensive if you put a bigger size.
    batch=16, # Tweak the batch size according to your GPU.
    name='yolov8x6-animeface', # Same as your comet project name
    project='yolov8x6-animeface', # Same as your comet project name
    device=[0], # Helpful if your GPU isn't recognized automatically.
    patience=0, # Tweak this as needed
    resume=False,
)

Few important points :

We're using the **default SGD optimizer** with automatic values. Adam and RMSProp have both been tried during training but weren't as good as SGD. They were mediocre, even when fidgeting with hyper-parameters.

Here is something we witness when using early stopping: 
- The model detects correctly, but it lacks confidence.
- Metrics are high (0.57 mAP50-95 on val set) but confidence is around 0.5 on every detection.
Here is what happens when the model trains 300 epochs : 
- Metrics get lower values and converge around mAP50-95 0.52, but confidence gets around 0.8 on every detection.

**If you have to stop the training for whatever reason and want to resume training :**
1. Replace path in line 4 by the path of the `last.pt` weight previously calculated.
2. Edit resume parameter to `resume=True,` inside the training arguments. Be aware that if you keep `resume=True` and try to train a **new** model, it will give you an unrelated CUDA error (Invalid CUDA requested).


In [ ]:
# End comet monitoring
experiment.end()

## Evaluation on test subset

If you've split your dataset properly, you can now do inference on your test sub-dataset.

In [ ]:
eval_model = YOLO('./yolov8x6-animeface/yolov8x6-animeface/weights/last.pt') # Adapt to your own path

metrics = eval_model.val(
    data='./datasets/data.yaml',
    split='test'
)